In [98]:
import pandas as pd

In [99]:
# Step 1: Read the Data
def read_data(filepath):
    """
    Reads data from a CSV or TXT file based on the file extension. Assuming a comma delimiter in both cases
    Args:
        filepath (str): Path to the data file.

    Returns:
        pd.DataFrame: The loaded data as a DataFrame.
    """
    if filepath.endswith(".csv") or filepath.endswith(".txt"):
        try:
            data = pd.read_csv(filepath)
        except:
            raise Exception("File not found.")
    else:
        raise Exception("Unsupported file format")
    return data

In [100]:
# Step 2: Data Validation
def validate_data(data):
    """
    Validates the data by checking missing values, outliers

    Args:
        data (pd.DataFrame): The data to be cleaned.

    Returns:
        pd.DataFrame: The cleaned data.
    """
    if "Airline" in data.columns:  # Assuming a column identifies the dataset
        return validate_airline_flights(data)
    elif "stock_symbol" in data.columns:
        return validate_stock_prices(data)
    else:
        raise Exception("Unsupported dataset")

def validate_airline_flights(data):
    """
    Validates the airline data to ensure it meets specific requirements.

    Args:
        data (pd.DataFrame): The data to be validated.

    Returns:
        bool: True if data is valid, False otherwise.
    """
    # Check for missing values
    airline_missing = data.isnull().sum()
    # Count and remove NaN data
    row_nan_count = data.isnull().any(axis=1).sum()
    print('Total row count: ', len(data))
    print('Rows with missing data', row_nan_count)
    print('Missing values in airline data:\n', airline_missing)
    if row_nan_count > (0.9*len(data)):
        return False
    return True
    
def validate_stock_prices(data):
    """
    Validates the stock data to ensure it meets specific requirements.

    Args:
        data (pd.DataFrame): The data to be validated.

    Returns:
        bool: True if data is valid, False otherwise.
    """
    # Check for missing values
    stock_missing = data.isnull().sum()
    row_nan_count = data.isnull().any(axis=1).sum()
    print('Total row count: ', len(data))
    print('Rows with missing data', row_nan_count)
    print('Missing values in stock data:\n', stock_missing)
    if row_nan_count > (0.9*len(data)):
        return False
    return True

In [101]:
#Step 3: Data Cleaning
def clean_data(data):
    """
    Cleans the data by handling missing values, outliers, and data type conversions.

    Args:
        data (pd.DataFrame): The data to be cleaned.

    Returns:
        pd.DataFrame: The cleaned data.
    """
    if "Airline" in data.columns:  # Assuming a column identifies the dataset
        return clean_airline_flights(data)
    elif "stock_symbol" in data.columns:
        return clean_stock_prices(data)
    else:
        raise Exception("Unsupported dataset")


def clean_airline_flights(data):
    """
    Cleans the airline flights dataset.

    Args:
        data (pd.DataFrame): The airline flights data to be cleaned.

    Returns:
        pd.DataFrame: The cleaned airline flights data.
    """
    clean_data = data.copy()
    clean_data = clean_data.dropna()
    
    # Convert data types
    try:
        clean_data['Date'] = pd.to_datetime(clean_data['Date'])
        clean_data[['Revenue ($)', 'Distance (km)']] = clean_data[['Revenue ($)', 'Distance (km)']].astype(float)
        clean_data[['Flight Number','Passengers (First Class)', 'Passengers (Business Class)', 'Passengers (Economy Class)']] = clean_data[['Flight Number','Passengers (First Class)', 'Passengers (Business Class)', 'Passengers (Economy Class)']].astype(int, errors='ignore')
        clean_data['Origin'] = clean_data['Origin'].str.upper()  # Assuming origin is a string column
        clean_data['Destination'] = clean_data['Destination'].str.upper()  # Assuming destination is a string column
        clean_data['Departure Gate'] = clean_data['Departure Gate'].str.upper()  # Assuming aircraft type is a string column
        clean_data['Arrival Gate'] = clean_data['Arrival Gate'].str.upper()  # Assuming aircraft type is a string column
        clean_data['Aircraft Type'] = clean_data['Aircraft Type'].astype(str)
        clean_data['Pod'] = clean_data['Pod'].str.upper() 
    except:
        raise Exception("One or more columns may have incorrectly formatted data.")
    return clean_data

def clean_stock_prices(data):
    """ Cleans the big tech stock prices dataset.

    Args:
        data (pd.DataFrame): The stock prices data to be cleaned.

    Returns:
        pd.DataFrame: The cleaned stock prices data.
    """
    clean_data = data.copy()
    clean_data = clean_data.dropna()
    # Handling Missing Values
    clean_data.fillna(method='ffill', inplace=True)  # Forward-fill to propagate last valid observation

    # Convert data types
    clean_data['date'] = pd.to_datetime(clean_data['date'])
    clean_data[['open', 'high', 'low', 'close', 'adj_close']] = clean_data[['open', 'high', 'low', 'close', 'adj_close']].astype(float)
    clean_data['volume'] = clean_data['volume'].astype(int) 
    return clean_data

In [102]:
# Step 4: Save data
def save_data(data, filepath):
    """
    Saves the data to a CSV format.

    Args:
      data (pd.DataFrame): The data to be saved.
      filepath (str): Path to the output file.
    """
    try:
        data.to_csv(filepath, index=False)
    except:
        raise Exception("Cannot save data to CSV")

In [103]:
# Define the data processing pipeline steps
def data_processing_pipeline(filepath):
    """
    Performs the data processing pipeline steps: read, validate, clean and save.

    Args:
        filepath (str): Path to the data file.
            
    """
    data = read_data(filepath)
    validated_data = validate_data(data)
    if validated_data:
        cleaned_data = clean_data(data)
        save_data(cleaned_data, "cleaned_" + filepath)
        print(f"Saved data at cleaned_{filepath}")
    else:
        print(f"Data validation failed for {filepath}")
         


In [104]:
#data = data_processing_pipeline("airline_flights.csv")
data_processing_pipeline("big_tech_stock_prices.txt")


Total row count:  45088
Rows with missing data 0
Missing values in stock data:
 stock_symbol    0
date            0
open            0
high            0
low             0
close           0
adj_close       0
volume          0
dtype: int64
   stock_symbol       date      open      high       low     close  adj_close  \
0          AAPL 2010-01-04  7.622500  7.660714  7.585000  7.643214   6.515213   
1          AAPL 2010-01-05  7.664286  7.699643  7.616071  7.656429   6.526476   
2          AAPL 2010-01-06  7.656429  7.686786  7.526786  7.534643   6.422664   
3          AAPL 2010-01-07  7.562500  7.571429  7.466071  7.520714   6.410790   
4          AAPL 2010-01-08  7.510714  7.571429  7.466429  7.570714   6.453412   
5          AAPL 2010-01-11  7.600000  7.607143  7.444643  7.503929   6.396483   
6          AAPL 2010-01-12  7.471071  7.491786  7.372143  7.418571   6.323721   
7          AAPL 2010-01-13  7.423929  7.533214  7.289286  7.523214   6.412922   
8          AAPL 2010-01-14  7.5039

In [22]:
data_processing_pipeline("airline_flights.csv")

Total row count:  4000
Rows with missing data 2507
Missing values in airline data:
 Date                             0
Departure Time                  13
Revenue ($)                     15
Passengers (First Class)        18
Passengers (Business Class)     21
Passengers (Economy Class)      17
Origin                         560
Destination                    584
Pod                             13
Distance (km)                   23
Flight Number                   13
Aircraft Type                   19
Departure Gate                 845
Arrival Gate                   856
Airline                        730
dtype: int64
         Date Departure Time  Revenue ($)  Passengers (First Class)  \
0  2023-06-19          01:37       2393.0                       1.0   
1  2023-06-27          04:10       2236.0                       9.0   
2  2023-06-08          07:44       2072.0                       9.0   
6  2023-07-03          23:34       1949.0                       7.0   
9  2023-07-01          

In [ ]:
print("Data processing pipeline completed!")